In [1]:
import pandas as pd
import pyam
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import aneris

In [2]:
fasst_concentration_path = '../../../data/SOD/model_results/raw/FASST_CONC10072024.xlsx'
fasst_exposure_path = '../../../data/SOD/model_results/raw/FASST_EXPO10072024.xlsx'

In [3]:
def round_to_0_1_percent(value):
    if value == 0:
        return 0
    # Determine the number of significant digits to round to
    magnitude = np.floor(np.log10(abs(value))) - 2
    rounding_factor = 10 ** magnitude
    return np.round(value / rounding_factor) * rounding_factor

In [4]:
df_unclean_conc = pd.read_excel(fasst_concentration_path)
df_unclean_expo = pd.read_excel(fasst_exposure_path)
df_unclean = pd.concat([df_unclean_conc, df_unclean_expo])

# df_unclean.columns = df_unclean.columns.str.lower()
df_unclean['SCENARIO'] = df_unclean['SCENARIO'].replace({'UN_LIFE': 'LIFE-TP-v2',
                                                   'UN_TECH': 'TECH-TP-v2',
                                                   'UN_REF':'REF-v2',
                                                   'UN_TECH-TP': 'TECH-TP-v2',
                                                   'UN_LIFE-TP': 'LIFE-TP-v2'
                                                  })
df_unclean['VARIABLE'] = df_unclean['VARIABLE'].replace({'Population|Exposure to Ambient PM2.5 > 25 ?g/m3':'Population|Exposure to Ambient PM2.5 > 25 μg/m3'})
df_unclean['REGION'] = df_unclean['REGION'].replace({
    'Africa': 'Africa (UN-R5)',
    'Asia and the Pacific': 'Asia and the Pacific (UN-R5)',
    'Eastern Europe': 'Eastern Europe (UN-R5)',
    'Latin American and Caribbean': 'Latin America and Caribbean (UN-R5)',
    'Western European and Others ': 'Western Europe and Other States (UN-R5)',
    'global':'World',
    'GLOBAL':'World'
                                                })

df_unclean

,MODEL,SCENARIO,REGION,VARIABLE,UNIT,2015,2020,2025,2030,2035,2040,2045,2050
0,JRC-FASST 1.0.0,REF-v2,Africa (UN-R5),Concentration|PM2.5,µg/m3,32.253383,32.769254,33.744327,34.345459,34.201908,34.252339,33.684594,33.832796
1,JRC-FASST 1.0.0,REF-v2,Asia and the Pacific (UN-R5),Concentration|PM2.5,µg/m3,35.135252,33.899814,35.927905,36.707457,35.372212,34.735411,32.062661,30.645238
2,JRC-FASST 1.0.0,REF-v2,Eastern Europe (UN-R5),Concentration|PM2.5,µg/m3,13.135932,11.983452,12.426190,12.605705,12.144018,11.743248,10.947290,10.628136
3,JRC-FASST 1.0.0,REF-v2,Latin America and Caribbean (UN-R5),Concentration|PM2.5,µg/m3,15.095752,14.662310,14.962843,15.115323,14.824444,14.700729,14.175144,14.017843
4,JRC-FASST 1.0.0,REF-v2,Western Europe and Other States (UN-R5),Concentration|PM2.5,µg/m3,7.498257,6.785517,6.982190,7.167541,7.090150,7.001158,6.744018,6.613139
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,JRC-FASST 1.0.0,TECH-TP-v2,Africa (UN-R5),Population|Exposure to Ambient PM2.5 > 25 μg/m3,%,56.647412,57.025756,51.260974,48.436180,48.234660,48.221744,46.972698,47.242762
32,JRC-FASST 1.0.0,TECH-TP-v2,Asia and the Pacific (UN-R5),Population|Exposure to Ambient PM2.5 > 25 μg/m3,%,66.492219,61.208338,49.446566,43.166204,41.691532,41.103313,39.708862,39.415201
33,JRC-FASST 1.0.0,TECH-TP-v2,Eastern Europe (UN-R5),Population|Exposure to Ambient PM2.5 > 25 μg/m3,%,3.051219,1.270975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
34,JRC-FASST 1.0.0,TECH-TP-v2,Latin America and Caribbean (UN-R5),Population|Exposure to Ambient PM2.5 > 25 μg/m3,%,7.047721,6.113556,3.114746,2.440566,2.166245,2.034764,2.000776,1.976840


In [5]:
def plot_data_3_x_3_figures(df, title_dict=None, path_dict=None, variables=None,):

    # Define the color palette for scenarios
    scenario_colors = {
        "REF": "black",
        "LIFE-TP": "orange",
        "TECH-TP": "magenta",
        "REF-v2": "black",
        "LIFE-TP-v2": "orange",
        "TECH-TP-v2": "magenta",
    }

    df_plotting = df.filter(year=[2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050]).timeseries().reset_index()
    df_compare_data = pd.melt(df_plotting, id_vars=['model',
                                'scenario',
                                'region',
                                'variable',
                                'unit'
                                    ], var_name='year', value_name='value')
    df_compare_data['year'] = df_compare_data['year'].astype(int)

    if variables is None:
        variables = df.variable

    for variable in variables:
        df_var = df_compare_data[df_compare_data["variable"] == variable]
        # Get unique regions for creating subplots
        unique_regions = df_var['region'].unique()

        # Create subplots based on the number of unique regions
        fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))

        # Initialize lists to store handles and labels for the legend
        legend_handles = []
        legend_labels = []

        # Iterate through each region and plot in respective subplot
        for idx, region in enumerate(unique_regions):
            row_idx = idx // 3
            col_idx = idx % 3
            
            ax = axes[row_idx, col_idx]  # Select the current subplot
            
            region_data = df_var[df_var['region'] == region]  # Filter data for the current region
            
            # Plot lineplots for each region
            lineplot = sns.lineplot(data=region_data, 
                                    x="year", 
                                    y="value", 
                                    hue="scenario", 
                                    # style='model', 
                                    palette=scenario_colors, 
                                    linewidth=2.5, 
                                    ax=ax)
            
            # Customize ticks and labels for the current subplot
            ax.set_ylabel(region_data["unit"].iloc[0], fontsize=14)  # Set the y-axis label to the unit
            # Set y-axis lower limit to 0
            ax.set_ylim(min(0,ax.get_ylim()[0]), ax.get_ylim()[1])
            ax.set_xlabel('', fontsize=1)
            ax.set_title(f"{region}", fontsize=14)  # Set the title to the region
            ax.set_xticks([2010, 2020, 2030, 2040, 2050])
            ax.set_xticklabels([2010, 2020, 2030, 2040, 2050], 
                            rotation=45, fontsize=14)
            
            # Increase fontsize of y-axis tick labels
            ax.tick_params(axis='y', labelsize=14)

            # Add the lineplot to the legend manually
            legend_handles.append(lineplot)
            legend_labels.append(region)
            # Disable the legend for individual subplots
            ax.legend().set_visible(False)
            ax.grid(True)

            # Add a gray background for the 'World' region subplot
            if region == 'World':
                ax.set_facecolor(color='#ededed')

        # Show legend and grid for the current subplot
        if title_dict:
            fig.suptitle(f"{title_dict[variable]}", fontsize=16)
        else:
            fig.suptitle(variable, fontsize=16)

        handles, labels = ax.get_legend_handles_labels()
        fig.legend(handles, labels, title="Legend", bbox_to_anchor=(1.12, 0.55))
        plt.tight_layout()  # Adjust subplots to prevent overlap
        
        if path_dict:
            root_variable = variable.split("|")[0].replace(" ", "_")
            variable_name = variable.replace(" > 25 μg/m3", "").replace("|", "_").replace(" ", "_").replace("/", "")
            # plt.savefig(f"../../../plots/SOD/chpt_20/Regional/{path_dict[variable]}/{variable_name}.png", bbox_inches='tight')
            # plt.close()

In [6]:
def plot_data_individual_figures(df, title_dict=None, path_dict=None, variables=None,):
    # Define the color palette for scenarios
    scenario_colors = {
        "REF": "black",
        "LIFE-TP": "orange",
        "TECH-TP": "magenta",
        "REF-v2": "black",
        "LIFE-TP-v2": "orange",
        "TECH-TP-v2": "magenta",
    }

    regions_dict = {
        'Africa (UN-R5)': 'Africa',
        'Asia and the Pacific (UN-R5)': 'AatP',
        'Eastern Europe (UN-R5)': 'EE',
        'Latin America and Caribbean (UN-R5)': 'LaaC',
        'Western Europe and Other States (UN-R5)': 'WeoS'
                }
    
    if variables is None:
            variables = df.variable

    for region in list(regions_dict.keys()):
        df_plotting = df.filter(year=[2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050], region=region).timeseries().reset_index()

        
        df_selected = df_plotting[df_plotting['variable'].isin(variables)]

        df_compare_data = pd.melt(df_selected, id_vars=['model',
                                    'scenario',
                                    'region',
                                    'variable',
                                    'unit'
                                        ], var_name='year', value_name='value')
        df_compare_data['year'] = df_compare_data['year'].astype(int)

        

        for variable in variables: # ,variables
            # Use Seaborn to create the plot with markers and lines
            data = df_compare_data[df_compare_data["variable"] == variable]

            fig, ax = plt.subplots(figsize=(10, 6))  # Set figure size
            sns.lineplot(
                data=data,
                x="year",
                y="value",
                hue="scenario",
                # style="model",
                dashes=True,
                palette=scenario_colors,
            )

            # Set labels and title
            plt.xlabel("Year")
            plt.ylabel(data["unit"].iloc[0])  # Set the y-axis label to the unit
            # Set y-axis lower limit to 0
            ax.set_ylim(min(0,ax.get_ylim()[0]), ax.get_ylim()[1])
            # Mid point of left and right x-positions
            mid = (fig.subplotpars.right + fig.subplotpars.left)/2
            if title_dict:
                plt.suptitle(title_dict[variable], x=0.4)  # Set the title to the variable
                plt.title(region)
            else:
                fig.suptitle(variable, fontsize=16)
            # Show legend and grid
            plt.legend(title="Legend", bbox_to_anchor=(1.3, 1))
            plt.grid(True)
            # Adjust layout to ensure legend fits within the saved image
            plt.tight_layout()
            # root_variable = variable.split("|")[0].replace(" ", "_")
            if path_dict:
                variable_name = variable.replace(" > 25 μg/m3", "").replace("|", "_").replace(" ", "_").replace("/", "")
                plt.savefig(
                    f"../../../plots/SOD/chpt_20/Regional/{regions_dict[region]}/{path_dict[variable]}/{variable_name}.png"
                )
                plt.close()

In [7]:
df_mid_clean = pyam.IamDataFrame(df_unclean)
df_copy = df_mid_clean.timeseries()
# Remove the model level from index
# df_copy = df_copy.reset_index(level='model', drop=True)
df_copy.columns = df_copy.columns.astype(str)

hist_filtered = df_mid_clean.filter(scenario='REF-v2', year=[2020]).timeseries()
# hist_filtered = hist_filtered.copy().reset_index(level='model', drop=True)
hist_filtered.columns = hist_filtered.columns.astype(str)

def change_first_level_index_value(df, old_value, new_value):
    new_index = df.index.to_frame()  # Convert index to DataFrame
    new_index.loc[new_index['scenario'] == old_value, 'scenario'] = new_value  # Change the values
    new_df = df.copy()
    new_df.index = pd.MultiIndex.from_frame(new_index)  # Convert back to MultiIndex
    return new_df

# Change 'A' to 'C' in the first level of the MultiIndex
life_hist = change_first_level_index_value(hist_filtered, 'REF-v2', 'LIFE-TP-v2')
tech_hist = change_first_level_index_value(hist_filtered, 'REF-v2', 'TECH-TP-v2')

hist_conc = pd.concat([hist_filtered, life_hist, tech_hist])
df_copy['2020'] = hist_conc['2020']
df_copy

2015  \
model           scenario   region         variable                                        unit                   
JRC-FASST 1.0.0 LIFE-TP-v2 Africa (UN-R5) Concentration|NOx                               µg/m3       3.865927   
                                          Concentration|Ozone                             ppb        37.757587   
                                          Concentration|PM2.5                             µg/m3      32.238752   
                                          Population                                      million  1175.671833   
                                          Population|Exposure to Ambient PM2.5 > 25 μg/m3 %          56.647412   
...                                                                                                        ...   
                TECH-TP-v2 World          Concentration|NOx                               µg/m3       8.280340   
                                          Concentration|Ozone                             ppb        41.541041   
                                          Concentration|PM2.5                             µg/m3      28.805096   
                                          Population                                      million  7424.862459   
                                          Population|Exposure to Ambient PM2.5 > 25 μg/m3 %          49.427143   

                                                                                                          2020  \
model           scenario   region         variable                                        unit                   
JRC-FASST 1.0.0 LIFE-TP-v2 Africa (UN-R5) Concentration|NOx                               µg/m3       4.024537   
                                          Concentration|Ozone                             ppb        37.838764   
                                          Concentration|PM2.5                             µg/m3      32.769254   
                                          Population                                      million  1304.310820   
                                          Population|Exposure to Ambient PM2.5 > 25 μg/m3 %          58.361768   
...                                                                                                        ...   
                TECH-TP-v2 World          Concentration|NOx                               µg/m3       8.327150   
                                          Concentration|Ozone                             ppb        42.052978   
                                          Concentration|PM2.5                             µg/m3      28.115271   
                                          Population                                      million  7799.681550   
                                          Population|Exposure to Ambient PM2.5 > 25 μg/m3 %          48.872927   

                                                                                                          2025  \
model           scenario   region         variable                                        unit                   
JRC-FASST 1.0.0 LIFE-TP-v2 Africa (UN-R5) Concentration|NOx                               µg/m3       3.224420   
                                          Concentration|Ozone                             ppb        36.379146   
                                          Concentration|PM2.5                             µg/m3      30.473281   
                                          Population                                      million  1435.355433   
                                          Population|Exposure to Ambient PM2.5 > 25 μg/m3 %          51.433553   
...                                                                                                        ...   
                TECH-TP-v2 World          Concentration|NOx                               µg/m3       6.004482   
                                          Concentration|Ozone                             ppb        40.061981   


In [8]:
df_clean = pyam.IamDataFrame(df_copy)
df_clean = df_clean.filter(variable=[
    'Concentration|NOx',
    'Concentration|Ozone',
    'Concentration|PM2.5',
    'Population|Exposure to Ambient PM2.5 > 25 μg/m3'])

df_clean = df_clean.rename(scenario={
    'REF-v2':'REF', 
    'TECH-TP-v2':'TECH-TP', 
    'LIFE-TP-v2':'LIFE-TP'})


path_dict = {
    'Concentration|NOx': 'Air_Pollution',
    'Concentration|Ozone': 'Air_Pollution',
    'Concentration|PM2.5': 'Air_Pollution',
    'Population|Exposure to Ambient PM2.5 > 25 μg/m3': 'Air_Pollution'}

title_dict = {
    'Concentration|NOx': 'Concentration NOx',
    'Concentration|Ozone': 'Concentration Ozone',
    'Concentration|PM2.5': 'Concentration PM2.5',
    'Population|Exposure to Ambient PM2.5 > 25 μg/m3': 'Population Exposed to Ambient PM2.5 > 25 μg/m3'}

plot_data_individual_figures(df_clean, title_dict=title_dict, path_dict=path_dict)
df_clean.to_excel('../../../data/SOD/model_results/to_share/FASST_to_share.xlsx')
# plot_data_3_x_3_figures(df_clean)